In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install fanalysis

from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])

In [ ]:

colonnes_montant = ["DEDUPLI", "IDENT", "SommeDeMONTANT_NUM", "Flag_Top_Montant", "NovosScore_Vingtile"]
colonnes_montant = colonnes_montant[:-1]

data_montant_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/DONS_AA_N_TODAY_1ers gestes_REF_MD.txt", sep=";", header = 0, low_memory=False, dtype=str) #, dtype=str )#, dtype='Int64')
data_montant_orig =data_montant_orig.iloc[:,1:]  

data_montant_orig.iloc[:,2]=data_montant_orig.iloc[:,2].str.replace(",",".")

data_montant_orig.columns =colonnes_montant
data_montant_orig["SommeDeMONTANT_NUM"]=data_montant_orig["SommeDeMONTANT_NUM"].astype(float)


In [ ]:
# importation du fichier (changer le chemin d'accès)
''' Faites attention au type de fichier, au délimiteur des valeurs (ça doit être unique et non multiple),
au header et à la lecture des types d'objets (low_memory) '''

data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/sep_AAPOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False,dtype=str )#, dtype='Int64')


In [ ]:
''' copie superficielle (peu profonde) du dataset original pour ne pas modifier les valeurs
d'origine par erreur plus tard 
'''
data = data_orig.copy()
data.shape

(152510, 23)

In [ ]:
# visualisation des premiers lignes
data.head()

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,0010000067,01,72,07,83,08,06,17,18,03,44,81,04,08,06,12,02,03,04,12,31,03,1
1,0010000070,01,72,07,84,08,05,18,20,01,33,44,03,04,08,08,05,09,05,20,36,03,2
2,0010000090,03,64,06,75,07,05,13,08,01,71,67,07,06,12,11,10,08,04,02,22,02,2
3,0010000124,03,43,04,82,08,01,15,17,03,51,81,05,08,10,12,10,14,05,15,61,06,2
4,0010000147,02,25,02,32,03,07,07,15,05,71,81,07,08,12,12,21,21,06,21,61,06,1


In [ ]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path): # data_orig est une variables locale (peut être différente au df d'origine)
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe Top_mon)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # vingtiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    #print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)

    # création de la dernière colonne CLASSE et son remplissage
    index_20_ord = pd.DataFrame(np.unique(df_tous["TRANCHES"])) 
    df_tous["CLASSE"]=df_tous["TRANCHES"].copy()
    ind_val=1
    for kk in range(20):
        df_tous["CLASSE"][df_tous["CLASSE"]== index_20_ord.iloc[kk,0] ] = str(ind_val) 
        ind_val+=1
    print(df_tous)    
    
    # pour la colonne Top_mon : changement de type de variable et remplissage des valeurs (s'il le faut)
    data_temp_t = data_orig.copy()
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t #data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne TOTAL
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

    return  df_tous


def tableau_5_montant(df_tous, data_montant_orig, affichage, save, path):
    # tableau avec les deux variables qui nous intéressent
    ident_et_montant = data_montant_orig[["IDENT","SommeDeMONTANT_NUM"]]

    # jointure
    tableau_resultats_et_montant = pd.merge(df_tous,ident_et_montant)
    #print(tableau_resultats_et_montant) 

    # on élimine toutes les sommes supérieur à 5000 €
    tableau_resultats_et_montant = tableau_resultats_et_montant[tableau_resultats_et_montant["SommeDeMONTANT_NUM"] <= 5000]
    print(tableau_resultats_et_montant) 
    #print(tableau_resultats_et_montant["TRANCHES"].value_counts())
    #print(pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])))

    # index
    index_temp = pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])) 

    # création du tableau qui contiendra les résultats finaux
    tableau_resultats_finaux = pd.DataFrame(columns=["TRANCHES","Moyen_Somme","TOTAL"],index=range(20))
    tableau_resultats_finaux["TRANCHES"] = index_temp

    # remplissage du tableau final
    for k in range(20):
        tableau_resultats_finaux.iloc[k,1] = tableau_resultats_et_montant[tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]]["SommeDeMONTANT_NUM"].mean()
        tableau_resultats_finaux.iloc[k,2] = (tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]).sum()
    
    # affichage
    if affichage:        
        print(tableau_resultats_finaux)
    
    # enregistrement sous Excel
    if save:
        tableau_resultats_finaux.to_excel(path)


def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data
          


In [ ]:
def vingtiles_xgb(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var].values)
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV, RandomizedSearchCV

In [ ]:
####################  DONNEES SANS LES VARIABLES CORR ET ENCODAGE DUMMIES        ###########################

In [96]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F","Onoma-Phase_Scl",
                "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
####################################################################################
#                            MODELE 1 : BOOSTING
####################################################################################

In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_
#model_AdaBoost.get_params


{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 4,
 'n_estimators': 50}

In [ ]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="entropy",max_depth=4),n_estimators = 50 , random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =False)

test set : 
[[ 3391  2714]
 [ 8552 15845]]
              precision    recall  f1-score   support

           1       0.28      0.56      0.38      6105
           2       0.85      0.65      0.74     24397

    accuracy                           0.63     30502
   macro avg       0.57      0.60      0.56     30502
weighted avg       0.74      0.63      0.67     30502

train set : 
[[14550  9784]
 [33107 64567]]
              precision    recall  f1-score   support

           1       0.31      0.60      0.40     24334
           2       0.87      0.66      0.75     97674

    accuracy                           0.65    122008
   macro avg       0.59      0.63      0.58    122008
weighted avg       0.76      0.65      0.68    122008

GRAPHIQUE NON DEMANDE


In [ ]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_AdaBoost, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Adaboost.xlsx")

             IDENT     PROBA
68693   0010175854  0.051322
19020   0010049365  0.051556
113742  0010286147  0.052343
62768   0010162397  0.052610
12310   0010033454  0.052812
...            ...       ...
44608   0010115879  0.678602
57631   0010146679  0.679044
63886   0010163618  0.680184
12241   0010033382  0.806274
117303  0010296719  0.806477

[152510 rows x 2 columns]
0.505319    127
0.520884     73
0.505784     55
0.508899     49
0.509265     47
           ... 
0.497595      1
0.502488      1
0.500932      1
0.497675      1
0.500231      1
Name: PROBA, Length: 136773, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


             IDENT     PROBA                        TRANCHES CLASSE
68693   0010175854  0.051322  (0.051219999999999995, 0.4918]      1
19020   0010049365  0.051556  (0.051219999999999995, 0.4918]      1
113742  0010286147  0.052343  (0.051219999999999995, 0.4918]      1
62768   0010162397  0.052610  (0.051219999999999995, 0.4918]      1
12310   0010033454  0.052812  (0.051219999999999995, 0.4918]      1
...            ...       ...                             ...    ...
44608   0010115879  0.678602                (0.5055, 0.8065]     20
57631   0010146679  0.679044                (0.5055, 0.8065]     20
63886   0010163618  0.680184                (0.5055, 0.8065]     20
12241   0010033382  0.806274                (0.5055, 0.8065]     20
117303  0010296719  0.806477                (0.5055, 0.8065]     20

[152510 rows x 4 columns]
Top_mon                            1     2  TOTAL
TRANCHES                                         
(0.051219999999999995, 0.4918]   229  7397   7626
(0.4918

In [ ]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/montant_Adaboost.xlsx")

             IDENT     PROBA  ... CLASSE SommeDeMONTANT_NUM
0       0010175854  0.051322  ...      1               10.0
1       0010049365  0.051556  ...      1               10.0
2       0010286147  0.052343  ...      1               50.0
3       0010162397  0.052610  ...      1               10.0
4       0010033454  0.052812  ...      1               10.0
...            ...       ...  ...    ...                ...
152505  0010115879  0.678602  ...     20               50.0
152506  0010146679  0.679044  ...     20               10.0
152507  0010163618  0.680184  ...     20              300.0
152508  0010033382  0.806274  ...     20              100.0
152509  0010296719  0.806477  ...     20              100.0

[152134 rows x 5 columns]
                          TRANCHES Moyen_Somme TOTAL
0   (0.051219999999999995, 0.4918]     25.3997  7626
1                 (0.4918, 0.4951]     30.8026  7623
2                 (0.4951, 0.4962]     31.9412  7625
3                 (0.4962, 0.4968]     34

In [ ]:
"""
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_AdaBoost, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_Adaboost.xlsx")
"""

'\n# INDIVIDUS SANS TARGET\nvingtiles_ind_sans_target(model = final_model_AdaBoost, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,\n            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_Adaboost.xlsx")\n'

In [ ]:
#############################################################
#                   MODELE 2 : RANDOM FOREST
#############################################################

In [92]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[12,15,16,17,18,19] ,'n_estimators' : [50,100,200] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_


{'criterion': 'gini', 'max_depth': 4, 'max_features': 19, 'n_estimators': 50}

In [97]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="gini", max_depth=4, max_features=19, n_estimators = 50, random_state=0) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 3930  2175]
 [11456 12941]]
              precision    recall  f1-score   support

           1       0.26      0.64      0.37      6105
           2       0.86      0.53      0.66     24397

    accuracy                           0.55     30502
   macro avg       0.56      0.59      0.51     30502
weighted avg       0.74      0.55      0.60     30502

train set : 
[[15750  8584]
 [45897 51777]]
              precision    recall  f1-score   support

           1       0.26      0.65      0.37     24334
           2       0.86      0.53      0.66     97674

    accuracy                           0.55    122008
   macro avg       0.56      0.59      0.51    122008
weighted avg       0.74      0.55      0.60    122008

GRAPHIQUE NON DEMANDE


In [98]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_RandomForestClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Random_Forest.xlsx")

             IDENT     PROBA
72862   0010187238  0.325862
140157  0010355278  0.325862
102096  0010259577  0.328730
34591   0010091470  0.331894
108662  0010273645  0.332341
...            ...       ...
88649   0010224735  0.810035
133303  0010334382  0.810035
35731   0010092729  0.810035
118023  0010297514  0.810035
110804  0010282873  0.810035

[152510 rows x 2 columns]
0.509898    4801
0.505284    1972
0.433337    1532
0.503639    1154
0.454494     967
            ... 
0.511379       1
0.444134       1
0.635703       1
0.403297       1
0.422410       1
Name: PROBA, Length: 31357, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


             IDENT     PROBA        TRANCHES CLASSE
72862   0010187238  0.325862  (0.325, 0.412]      1
140157  0010355278  0.325862  (0.325, 0.412]      1
102096  0010259577  0.328730  (0.325, 0.412]      1
34591   0010091470  0.331894  (0.325, 0.412]      1
108662  0010273645  0.332341  (0.325, 0.412]      1
...            ...       ...             ...    ...
88649   0010224735  0.810035   (0.586, 0.81]     20
133303  0010334382  0.810035   (0.586, 0.81]     20
35731   0010092729  0.810035   (0.586, 0.81]     20
118023  0010297514  0.810035   (0.586, 0.81]     20
110804  0010282873  0.810035   (0.586, 0.81]     20

[152510 rows x 4 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.325, 0.412]   631  7108   7739
(0.412, 0.423]   804  6733   7537
(0.423, 0.43]    902  6709   7611
(0.43, 0.436]   1033  6583   7616
(0.436, 0.444]  1129  6496   7625
(0.444, 0.451]  1154  6479   7633
(0.451, 0.46]   1237  6389   7626
(0.46, 0.476]   1258  6360   7618
(0.476, 0

In [99]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/montant_Random_Forest.xlsx")

             IDENT     PROBA        TRANCHES CLASSE  SommeDeMONTANT_NUM
0       0010187238  0.325862  (0.325, 0.412]      1                40.0
1       0010355278  0.325862  (0.325, 0.412]      1                10.0
2       0010259577  0.328730  (0.325, 0.412]      1                20.0
3       0010091470  0.331894  (0.325, 0.412]      1                10.0
4       0010273645  0.332341  (0.325, 0.412]      1                10.0
...            ...       ...             ...    ...                 ...
152504  0010236918  0.810035   (0.586, 0.81]     20              5000.0
152505  0010224735  0.810035   (0.586, 0.81]     20                35.0
152507  0010092729  0.810035   (0.586, 0.81]     20              1500.0
152508  0010297514  0.810035   (0.586, 0.81]     20              1000.0
152509  0010282873  0.810035   (0.586, 0.81]     20              2000.0

[152134 rows x 5 columns]
          TRANCHES Moyen_Somme TOTAL
0   (0.325, 0.412]     28.5076  7737
1   (0.412, 0.423]     35.2774  753

In [ ]:
"""
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_RandomForestClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_Random_Forest.xlsx")
"""

In [ ]:
#############################################################
#                   MODELE 3 : BAGGING
#############################################################

In [120]:
model_Bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 1)), 'n_estimators' : [20,50,100],
         'bootstrap':[True], "bootstrap_features": [False]}

grid_Bagging = RandomizedSearchCV(model_Bagging, param, cv = 4, scoring = "recall", n_jobs=-1, random_state=0)
resultat_grid_Bagging = grid_Bagging.fit(train_X_s, train_Y_s)
resultat_grid_Bagging.best_params_


{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'bootstrap': True,
 'bootstrap_features': False,
 'n_estimators': 100}

In [121]:
final_model_BaggingClassifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced", criterion="gini", max_depth=2, random_state=0),bootstrap=True, bootstrap_features=False, n_estimators = 100) 
evaluation(final_model_BaggingClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 4593  1512]
 [15122  9275]]
              precision    recall  f1-score   support

           1       0.23      0.75      0.36      6105
           2       0.86      0.38      0.53     24397

    accuracy                           0.45     30502
   macro avg       0.55      0.57      0.44     30502
weighted avg       0.73      0.45      0.49     30502

train set : 
[[18372  5962]
 [60696 36978]]
              precision    recall  f1-score   support

           1       0.23      0.75      0.36     24334
           2       0.86      0.38      0.53     97674

    accuracy                           0.45    122008
   macro avg       0.55      0.57      0.44    122008
weighted avg       0.74      0.45      0.49    122008

GRAPHIQUE NON DEMANDE


In [122]:
vingtiles(model = final_model_BaggingClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Decision_Tree_Classifier.xlsx")

             IDENT     PROBA
43705   0010114902  0.393100
122355  0010308883  0.393100
71823   0010182453  0.393100
71822   0010182452  0.393100
71821   0010182451  0.393100
...            ...       ...
48772   0010127167  0.720654
22757   0010058366  0.720654
71630   0010180877  0.720654
40373   0010104669  0.720654
34688   0010091581  0.720654

[152510 rows x 2 columns]
0.510965    79121
0.393100    53727
0.710879     6927
0.598501     5128
0.720654     2829
0.608276     2662
0.717002      342
0.604624      340
0.700354      298
0.602153      237
0.587976      218
0.714531      213
0.597751      206
0.710129      200
0.594099       19
0.706477       17
0.591628       13
0.704006       13
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
#############################################################
#                   MODELE 4 : EXTRA-TREES
#############################################################

In [ ]:
model_ExtraTreesClassifier = ExtraTreesClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100] }# 100

grid_ExtraTreesClassifier = GridSearchCV(model_ExtraTreesClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_ExtraTreesClassifier = grid_ExtraTreesClassifier.fit(train_X_s, train_Y_s)
resultat_grid_ExtraTreesClassifier.best_params_



{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 12,
 'n_estimators': 20}

In [ ]:
final_model_ExtraTreesClassifier = ExtraTreesClassifier( criterion="entropy", max_depth=2, max_features=12, n_estimators=20 , class_weight="balanced", random_state=0) 
evaluation(final_model_ExtraTreesClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 4023  2082]
 [12109 12288]]
              precision    recall  f1-score   support

           1       0.25      0.66      0.36      6105
           2       0.86      0.50      0.63     24397

    accuracy                           0.53     30502
   macro avg       0.55      0.58      0.50     30502
weighted avg       0.73      0.53      0.58     30502

train set : 
[[16180  8154]
 [48407 49267]]
              precision    recall  f1-score   support

           1       0.25      0.66      0.36     24334
           2       0.86      0.50      0.64     97674

    accuracy                           0.54    122008
   macro avg       0.55      0.58      0.50    122008
weighted avg       0.74      0.54      0.58    122008

GRAPHIQUE NON DEMANDE


In [ ]:
vingtiles_tableau=vingtiles(model = final_model_ExtraTreesClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Extra_Trees_Classifier.xlsx")

            IDENT     PROBA
24030  0010066445  0.384846
50943  0010129561  0.385199
2794   0010009664  0.385323
4828   0010013746  0.385519
7151   0010021053  0.385519
...           ...       ...
1613   0010008375  0.638298
98557  0010249184  0.638298
35494  0010092465  0.638298
95927  0010246296  0.642986
9416   0010023563  0.646302

[152510 rows x 2 columns]
0.507173    10746
0.460710     6072
0.502855     4957
0.510775     4063
0.469192     3708
            ...  
0.522516        1
0.471306        1
0.532955        1
0.476590        1
0.512777        1
Name: PROBA, Length: 6001, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            IDENT     PROBA        TRANCHES CLASSE
24030  0010066445  0.384846  (0.384, 0.455]      1
50943  0010129561  0.385199  (0.384, 0.455]      1
2794   0010009664  0.385323  (0.384, 0.455]      1
4828   0010013746  0.385519  (0.384, 0.455]      1
7151   0010021053  0.385519  (0.384, 0.455]      1
...           ...       ...             ...    ...
1613   0010008375  0.638298  (0.548, 0.646]     20
98557  0010249184  0.638298  (0.548, 0.646]     20
35494  0010092465  0.638298  (0.548, 0.646]     20
95927  0010246296  0.642986  (0.548, 0.646]     20
9416   0010023563  0.646302  (0.548, 0.646]     20

[152510 rows x 4 columns]
Top_mon            1      2  TOTAL
TRANCHES                          
(0.384, 0.455]   682   6974   7656
(0.455, 0.461]  1527  11023  12550
(0.461, 0.462]   375   2740   3115
(0.462, 0.467]  1193   7439   8632
(0.467, 0.471]   900   5283   6183
(0.471, 0.475]  1257   6698   7955
(0.475, 0.48]   1158   6163   7321
(0.48, 0.488]   1301   6756   8057
(0.488, 0.4

In [ ]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/montant_Extra_Trees_Classifier.xlsx")

             IDENT     PROBA        TRANCHES CLASSE  SommeDeMONTANT_NUM
0       0010066445  0.384846  (0.384, 0.455]      1                50.0
1       0010129561  0.385199  (0.384, 0.455]      1                10.0
2       0010009664  0.385323  (0.384, 0.455]      1                10.0
3       0010013746  0.385519  (0.384, 0.455]      1                10.0
4       0010021053  0.385519  (0.384, 0.455]      1                10.0
...            ...       ...             ...    ...                 ...
152505  0010008375  0.638298  (0.548, 0.646]     20                50.0
152506  0010249184  0.638298  (0.548, 0.646]     20               100.0
152507  0010092465  0.638298  (0.548, 0.646]     20                20.0
152508  0010246296  0.642986  (0.548, 0.646]     20               100.0
152509  0010023563  0.646302  (0.548, 0.646]     20                50.0

[152134 rows x 5 columns]
          TRANCHES Moyen_Somme  TOTAL
0   (0.384, 0.455]     28.7295   7654
1   (0.455, 0.461]     34.9231  1

In [ ]:
"""
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_ExtraTreesClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_Extra_Trees_Classifier.xlsx")
"""

'\n# INDIVIDUS SANS TARGET\nvingtiles_ind_sans_target(model = final_model_ExtraTreesClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,\n            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_Extra_Trees_Classifier.xlsx")\n'

In [ ]:
#############################################################
#                   MODELE 5 : XGBoost (IL FAUT L'AMELIORER CAR GRIDSEARCHCV PREND TROP DE TEMPS)
#############################################################

In [116]:

model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=-1, seed=0)
param = { 'scale_pos_weight': list(np.linspace(0.30, 0.40, 20)), 'max_depth':range(3,10,1), 'min_child_weight':range(1,6,1) } #list(np.linspace(0.10, 0.30, 2))   #'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2),

grid_XGBClassifier = RandomizedSearchCV(estimator = model_XGBClassifier,param_distributions=param, scoring='recall', cv=4, n_jobs=-1, random_state=0)
resultat_grid_XGBClassifier = grid_XGBClassifier.fit(train_X_s, train_Y_s)
resultat_grid_XGBClassifier.best_params_

{'max_depth': 3, 'min_child_weight': 3, 'scale_pos_weight': 0.3}

In [117]:
final_model_XGBClassifier = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=3, min_child_weight=3, gamma=0, subsample=0.8,
                                                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=-1,
                                           scale_pos_weight=0.3, seed=0)
evaluation(final_model_XGBClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 2332  3773]
 [ 4700 19697]]
              precision    recall  f1-score   support

           1       0.33      0.38      0.36      6105
           2       0.84      0.81      0.82     24397

    accuracy                           0.72     30502
   macro avg       0.59      0.59      0.59     30502
weighted avg       0.74      0.72      0.73     30502

train set : 
[[ 9515 14819]
 [18609 79065]]
              precision    recall  f1-score   support

           1       0.34      0.39      0.36     24334
           2       0.84      0.81      0.83     97674

    accuracy                           0.73    122008
   macro avg       0.59      0.60      0.59    122008
weighted avg       0.74      0.73      0.73    122008

GRAPHIQUE NON DEMANDE


In [118]:
vingtiles_xgb(model = final_model_XGBClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_XGBoost.xlsx")

             IDENT     PROBA
57037   0010142861  0.075088
151322  0010381076  0.075094
7384    0010021334  0.081045
106233  0010270932  0.081045
51014   0010129638  0.084102
...            ...       ...
52207   0010130991  0.828823
117250  0010296662  0.828823
95927   0010246296  0.830353
146030  0010368580  0.830353
9416    0010023563  0.837772

[152510 rows x 2 columns]
0.460342    1017
0.463859     484
0.471380     349
0.444466     321
0.348848     317
            ... 
0.449485       1
0.324482       1
0.449479       1
0.324478       1
0.375001       1
Name: PROBA, Length: 74455, dtype: int64
                 PROBA
57037   (0.0741, 0.28]
151322  (0.0741, 0.28]
7384    (0.0741, 0.28]
106233  (0.0741, 0.28]
51014   (0.0741, 0.28]
...                ...
52207   (0.648, 0.838]
117250  (0.648, 0.838]
95927   (0.648, 0.838]
146030  (0.648, 0.838]
9416    (0.648, 0.838]

[152510 rows x 1 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.0741, 0.28]   482  7144

In [ ]:
###################################################################################################################@
###################################################################################################################@
#                                   DECISION TREE CLASSIFIER  ----  AVEC VARIABLES QUALITATIVES
###################################################################################################################@
###################################################################################################################@

In [ ]:
####################################################################################
####################################################################################
#                   DECISION TREE
####################################################################################
####################################################################################

In [ ]:

model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced" )
param = { 'max_depth' :list(range(2,20)), 'criterion': ["gini", "entropy"], 'max_features' : list(range(3,10,1)),
         'min_samples_split' : [0.05, 0.1, 0.4, 0.6, 0.8, 1] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_


{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 8,
 'min_samples_split': 0.05}

In [ ]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, criterion="gini", class_weight = "balanced", max_depth = 2, max_features=8, min_samples_split=0.05 )
evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 5958   147]
 [23375  1022]]
              precision    recall  f1-score   support

           1       0.20      0.98      0.34      6105
           2       0.87      0.04      0.08     24397

    accuracy                           0.23     30502
   macro avg       0.54      0.51      0.21     30502
weighted avg       0.74      0.23      0.13     30502

train set : 
[[23782   552]
 [93668  4006]]
              precision    recall  f1-score   support

           1       0.20      0.98      0.34     24334
           2       0.88      0.04      0.08     97674

    accuracy                           0.23    122008
   macro avg       0.54      0.51      0.21    122008
weighted avg       0.74      0.23      0.13    122008

GRAPHIQUE NON DEMANDE


In [ ]:
# TRAIN SET TEST SET
vingtiles(model = final_model_DecisionTreeClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Decision_Tree_Classifier.xlsx")

             IDENT     PROBA
82633   0010211416  0.356121
60396   0010153141  0.356121
125380  0010319105  0.356121
9817    0010026095  0.356121
142896  0010358384  0.356121
...            ...       ...
85595   0010214677  0.566034
55583   0010141240  0.566034
92234   0010235374  0.566034
18221   0010046701  0.566034
137891  0010346067  0.566034

[152510 rows x 2 columns]
0.503490    144067
0.356121      5727
0.566034      2716
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
####################################################################################################
#                           SANS LES MODALITES REPRESENTANT LES VALEURS MANQUANTES
####################################################################################################

In [123]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"], axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )

colonnes_drop_nan = ['TypeHabi_04','GeoFaz_Scl_81','Anciennete_11','Logement_prix_m2_21','Mediane_RFR_21','Sexe_05','Prenom_Phase_scl_H_71','Prenom_Phase_scl_F_81','Age_H_12','Age_F_12','Affinite_1_21','Affinite_2_21','Prenom_effectif_06','Potentiel_prenom_21']
#colonnes_drop_nan=data.drop(colonnes_drop_nan)
data_1_22 = data_1_22.drop(colonnes_drop_nan, axis=1)


data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe Top_mon
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [124]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50] }

grid_AdaBoost = RandomizedSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall", n_jobs=-1, random_state=0)
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_
#model_AdaBoost.get_params


{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 4,
 'n_estimators': 20}

In [126]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="entropy",max_depth=4),n_estimators = 20 , random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =False)

test set : 
[[ 3360  2745]
 [ 8514 15883]]
              precision    recall  f1-score   support

           1       0.28      0.55      0.37      6105
           2       0.85      0.65      0.74     24397

    accuracy                           0.63     30502
   macro avg       0.57      0.60      0.56     30502
weighted avg       0.74      0.63      0.67     30502

train set : 
[[14100 10234]
 [33968 63706]]
              precision    recall  f1-score   support

           1       0.29      0.58      0.39     24334
           2       0.86      0.65      0.74     97674

    accuracy                           0.64    122008
   macro avg       0.58      0.62      0.57    122008
weighted avg       0.75      0.64      0.67    122008

GRAPHIQUE NON DEMANDE


In [127]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_AdaBoost, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Random_Forest.xlsx")

             IDENT     PROBA
37109   0010094324  0.000622
136822  0010344839  0.000705
9862    0010027044  0.000715
84659   0010213623  0.003589
45339   0010116687  0.003779
...            ...       ...
136764  0010344779  0.859214
103709  0010261346  0.859907
94362   0010237726  0.860608
32417   0010082460  0.862181
79429   0010201233  0.866046

[152510 rows x 2 columns]
0.500476    1410
0.499185     670
0.494007     536
0.499313     507
0.501239     389
            ... 
0.503290       1
0.497400       1
0.502104       1
0.493358       1
0.497205       1
Name: PROBA, Length: 49801, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


             IDENT     PROBA           TRANCHES CLASSE
37109   0010094324  0.000622  (0.000522, 0.485]      1
136822  0010344839  0.000705  (0.000522, 0.485]      1
9862    0010027044  0.000715  (0.000522, 0.485]      1
84659   0010213623  0.003589  (0.000522, 0.485]      1
45339   0010116687  0.003779  (0.000522, 0.485]      1
...            ...       ...                ...    ...
136764  0010344779  0.859214    (0.5127, 0.866]     20
103709  0010261346  0.859907    (0.5127, 0.866]     20
94362   0010237726  0.860608    (0.5127, 0.866]     20
32417   0010082460  0.862181    (0.5127, 0.866]     20
79429   0010201233  0.866046    (0.5127, 0.866]     20

[152510 rows x 4 columns]
Top_mon               1     2  TOTAL
TRANCHES                            
(0.000522, 0.485]   391  7235   7626
(0.485, 0.4891]     702  6923   7625
(0.4891, 0.4911]    813  6984   7797
(0.4911, 0.4927]    857  6597   7454
(0.4927, 0.494]    1007  6621   7628
(0.494, 0.495]     1074  6549   7623
(0.495, 0.496]   

In [104]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[12,15,16,17,18,19] ,'n_estimators' : [50,100,200] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=-1)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_


{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 16,
 'n_estimators': 100}

In [105]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="entropy", max_depth=2, max_features=16, n_estimators = 100, random_state=0) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 3763  2342]
 [10930 13467]]
              precision    recall  f1-score   support

           1       0.26      0.62      0.36      6105
           2       0.85      0.55      0.67     24397

    accuracy                           0.56     30502
   macro avg       0.55      0.58      0.52     30502
weighted avg       0.73      0.56      0.61     30502

train set : 
[[15245  9089]
 [43910 53764]]
              precision    recall  f1-score   support

           1       0.26      0.63      0.37     24334
           2       0.86      0.55      0.67     97674

    accuracy                           0.57    122008
   macro avg       0.56      0.59      0.52    122008
weighted avg       0.74      0.57      0.61    122008

GRAPHIQUE NON DEMANDE


In [106]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model = final_model_RandomForestClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_Random_Forest.xlsx")

             IDENT     PROBA
40449   0010104763  0.397721
107629  0010272515  0.399138
84690   0010213656  0.399716
14706   0010042800  0.401410
27516   0010070320  0.401410
...            ...       ...
114780  0010293934  0.597104
60898   0010153682  0.597104
25957   0010068572  0.597104
63660   0010163385  0.597104
86033   0010215848  0.597104

[152510 rows x 2 columns]
0.504448    2632
0.504978    1028
0.503089     825
0.499326     775
0.484389     752
            ... 
0.493821       1
0.477013       1
0.508937       1
0.487396       1
0.465631       1
Name: PROBA, Length: 47812, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


             IDENT     PROBA        TRANCHES CLASSE
40449   0010104763  0.397721  (0.397, 0.458]      1
107629  0010272515  0.399138  (0.397, 0.458]      1
84690   0010213656  0.399716  (0.397, 0.458]      1
14706   0010042800  0.401410  (0.397, 0.458]      1
27516   0010070320  0.401410  (0.397, 0.458]      1
...            ...       ...             ...    ...
114780  0010293934  0.597104   (0.54, 0.597]     20
60898   0010153682  0.597104   (0.54, 0.597]     20
25957   0010068572  0.597104   (0.54, 0.597]     20
63660   0010163385  0.597104   (0.54, 0.597]     20
86033   0010215848  0.597104   (0.54, 0.597]     20

[152510 rows x 4 columns]
Top_mon            1     2  TOTAL
TRANCHES                         
(0.397, 0.458]   633  6993   7626
(0.458, 0.463]   832  6796   7628
(0.463, 0.469]   986  6639   7625
(0.469, 0.477]   970  6653   7623
(0.477, 0.482]  1002  6625   7627
(0.482, 0.485]  1198  6801   7999
(0.485, 0.487]  1218  6056   7274
(0.487, 0.491]  1284  6318   7602
(0.491, 0

In [95]:
####################################################################################################
#                           SANS LES MODALITES AVEC PETITS EFFECTIFS
####################################################################################################

In [128]:
4183/7598

0.5505396156883391